In [9]:
# Test case
file = codecs.open("article_0.html", "r", "utf-8")
soup = BeautifulSoup(file, 'html.parser')
soup.find_all('strong')[0].contents[0]

'Fullmetal Alchemist: Brotherhood'

In [52]:
animeTitle = []
animeType = []
animeNumEpisode = []
releaseDate = []
endDate = []
animeNumMembers = []
animeScore = []
animeUsers = []
animeRank = []
animePopularity = []
animeDescription = []
animeRelated = []
animeCharacters = []
animeVoices = []
animeStaff = []


# I still don't know how to integrate directory paths
# The following code should work only with files
# Attention: There are some bugs 
for i in range(5):
    # anime titles as test case above
    path = "article_" + str(i) + ".html"
    file = codecs.open(path, "r", "utf-8")
    soup = BeautifulSoup(file, 'html.parser')
    animeTitle.append(soup.find_all('strong')[0].contents[0])
    
    # left of the html page
    # e.g. for article_0 under Alternative Titles we have English: Fullmetal Alchemist: Brotherhood 
    # that corresponds to 
    # <div class="spaceit_pad"> Fullmetal Alchemist: Brotherhood <span class="dark_text">English:</span></div>
    divs = soup.find_all("div", {"class": "spaceit_pad"})
    for div in divs:
        spans = div.find_all("span")
        for span in spans:
            # types
            if span.contents[0] == 'Type:':
                animeType.append(div.find_all('a')[0].contents[0])
            # number of episodes
            if span.contents[0] == 'Episodes:':
                animeNumEpisode.append(int(div.contents[2]))
            # release date and end date
            if span.contents[0] == 'Aired:':
                if len(div.contents[2]) > 21:
                    releaseDate.append(pd.to_datetime(div.contents[2][4:16], errors ='coerce', format='%m%d%Y'))
                    endDate.append(pd.to_datetime(div.contents[2][19:-4], errors ='coerce', format='%m%d%Y'))
                else:
                    releaseDate.append(pd.to_datetime(div.contents[2][4:-4], errors ='coerce', format='%m%d%Y'))
                    endDate.append('-')
                    
    # center of the html page
    # similar to what was done before
    divs = soup.find_all("div", {"class": "stats-block po-r clearfix"})
    for div in divs:
        # number of members
        members = div.find_all("span", {"class": "numbers members"})
        animeNumMembers.append(int(members[0].contents[1].contents[0].replace(',', '')))
        # score
        score = div.find_all("div", {"class": "statistics-info info1"})
        #animeScore.append(float(score[0].contents[0])) IndexError: list index out of range WHY???
        # number of users
        users = div.find_all("div", {"class": "fl-l score"})
        # here we we eliminate the word 'user '   
        # that is why there is the [:-6] part
        # we also replace the comma divisor
        animeUsers.append(int(users[0]['data-user'][:-6].replace(',', '')))
        # rank
        rank = div.find_all("span", {"class": "numbers ranked"})
        animeRank.append(int(rank[0].contents[1].contents[0][1:]))
        # popularity
        popularity = div.find_all("span", {"class": "numbers popularity"})
        animePopularity.append(int(popularity[0].contents[1].contents[0][1:]))
    
    # description
    description = soup.find_all("p", {"itemprop": "description"})
    for br in description[0].find_all("br"):
        #separate paragraphs in the synopsis with a new line
        br.replace_with("\n")
        #then animeDescription.append(text) I don't know how to do it!
        
    # related  
    x = []
    y = []
    related = soup.find_all("table", {"class": "anime_detail_related_anime"})
    for tr in related:
        td = tr.find_all("td")
        for i in range(0, len(td), 2):
            x.append(td[i].contents[0])
            t = td[i+1].find_all("a")
            y.append(t[0].contents[0])
        animeRelated.append('\n'.join([f'{x} {y}' for x, y in dict(zip(x, y)).items()]).split('\n'))
        
    # characters and voices
    characters = soup.find_all("div", {"class": "detail-characters-list clearfix"})
    chars = characters[0].find_all("h3", {"class": "h3_characters_voice_actors"})
    x = []
    for i in chars:
        x.append(i.contents[0].contents[0])
    animeCharacters.append(x)
    
    #voices
    voices = characters[0].find_all("td", {"class": "va-t ar pl4 pr4"})
    y = []
    for i in voices:
        y.append(i.contents[1].contents[0])
    animeVoices.append(y)

    # staff
    staff = soup.find_all("div", {"class": "detail-characters-list clearfix"})
    staff = staff[1].find_all("td")
    x = []
    y = []
    for i in range(1, len(staff), 2):
        x.append(staff[i].contents[1].contents[0])
        y.append(staff[i].find_all("small")[0].contents[0])
    animeStaff.append([list(i) for i in list(zip(x,y))])    

# check        
print(animeUsers)
print(animePopularity)
print(animeDescription)
print(animeRelated)
print(animeCharacters)
print(animeVoices)
print(animeStaff)

[1622384, 169476, 1087519, 1109700, 113310]
[3, 337, 33, 11, 651]
[]
[['Adaptation: Fullmetal Alchemist', 'Alternative version: Fullmetal Alchemist', 'Side story: Fullmetal Alchemist: Brotherhood Specials', 'Spin-off: Fullmetal Alchemist: Brotherhood - 4-Koma Theater'], ['Adaptation: Gintama', 'Prequel: Gintama Movie 2: Kanketsu-hen - Yorozuya yo Eien Nare', 'Side story: Gintama°: Umai-mono wa Atomawashi ni Suru to Yokodorisareru kara Yappari Saki ni Kue', 'Sequel: Gintama.'], ['Adaptation: Shingeki no Kyojin', 'Prequel: Shingeki no Kyojin Season 3', 'Sequel: Shingeki no Kyojin: The Final Season', 'Summary: Shingeki no Kyojin: Chronicle'], ['Adaptation: Steins;Gate', 'Alternative setting: ChäoS;HEAd', 'Sequel: Steins;Gate: Oukoubakko no Poriomania', 'Other: Steins;Gate: Soumei Eichi no Cognitive Computing', 'Alternative version: Steins;Gate: Kyoukaimenjou no Missing Link - Divide By Zero'], ['Adaptation: Fruits Basket', 'Prequel: Fruits Basket 2nd Season', 'Summary: Fruits Basket: Prel

In [54]:
df = pd.DataFrame(
    [animeTitle, animeType, animeNumEpisode, releaseDate, endDate, animeNumMembers, 
     animeScore, animeUsers, animeRank, animePopularity, animeDescription, animeRelated, 
     animeCharacters, animeVoices, animeStaff], 
    index=['Title', 'Type', 'Episodes','Release date', 'End date', 'Members', 'Score', 
           'Users', 'Rank', 'Popularity', 'Description', 'Related', 'Characters', 'Voices', 'Staff']).T
df.head()

,Title,Type,Episodes,Release date,End date,Members,Score,Users,Rank,Popularity,Description,Related,Characters,Voices,Staff
0,Fullmetal Alchemist: Brotherhood,TV,64,NaT,NaT,2675751,None,1622384,1,3,None,"[Adaptation: Fullmetal Alchemist, Alternative ...","[Elric, Edward, Elric, Alphonse, Mustang, Roy,...","[Park, Romi, Kugimiya, Rie, Miki, Shinichiro, ...","[[Cook, Justin, Producer], [Yonai, Noritomo, P..."
1,Gintama°,TV,51,NaT,NaT,483807,None,169476,2,337,None,"[Adaptation: Gintama, Prequel: Gintama Movie 2...","[Sakata, Gintoki, Kagura, Shimura, Shinpachi, ...","[Sugita, Tomokazu, Kugimiya, Rie, Sakaguchi, D...","[[Fujita, Youichi, Director, Storyboard, Plann..."
2,Shingeki no Kyojin Season 3 Part 2,TV,10,NaT,NaT,1596039,None,1087519,3,33,None,"[Adaptation: Shingeki no Kyojin, Prequel: Shin...","[Levi, Yeager, Eren, Ackerman, Mikasa, Arlert,...","[Kamiya, Hiroshi, Kaji, Yuki, Ishikawa, Yui, I...","[[Yabuta, Shuuhei, Producer], [Wada, Jouji, Pr..."
3,Steins;Gate,TV,24,NaT,NaT,2090910,None,1109700,4,11,None,"[Adaptation: Steins;Gate, Alternative setting:...","[Okabe, Rintarou, Makise, Kurisu, Shiina, Mayu...","[Miyano, Mamoru, Imai, Asami, Hanazawa, Kana, ...","[[Iwasa, Gaku, Producer], [Yasuda, Takeshi, Pr..."
4,Fruits Basket: The Final,TV,13,NaT,NaT,275214,None,113310,5,651,None,"[Adaptation: Fruits Basket, Prequel: Fruits Ba...","[Souma, Kyou, Honda, Tooru, Souma, Yuki, Souma...","[Uchida, Yuuma, Iwami, Manaka, Shimazaki, Nobu...","[[Ibata, Yoshihide, Director], [Aketagawa, Jin..."
